**合併數據**

In [9]:
import pandas as pd
import glob
import os

folder_path = './Training data/'
files = glob.glob(os.path.join(folder_path, '*.csv'))
data = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

In [ ]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1290889,9,2024-07-23 15:50:57.000,0.0,994.54,30.69,72.91,2288.33,1.10
1290890,9,2024-07-23 15:51:57.000,0.0,994.40,30.27,73.16,3236.67,1.92
1290891,9,2024-07-23 15:52:57.000,0.0,994.39,29.90,72.51,4526.67,3.57
1290892,9,2024-07-23 15:53:57.000,0.0,994.40,29.38,73.23,4231.67,3.13


**異常值處理**

**WindSpeed(m/s)**

**SunLight(Lux)**

In [11]:
from sklearn.model_selection import train_test_split

# 特徵與標籤
X = data[['Sunlight(Lux)']]  # 光照度
y = data['Power(mW)']       # 發電量

# 分割訓練與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression

# 建立線性回歸模型
model = LinearRegression()

# 訓練模型
model.fit(X_train, y_train)

# 獲取模型係數與截距
print(f"模型係數: {model.coef_[0]}")  # 光照度對發電量的影響係數
print(f"模型截距: {model.intercept_}")  # 發電量基線


模型係數: 0.015636212388786923
模型截距: -85.31809827114148


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# 預測
y_pred = model.predict(X_test)

# 計算誤差與解釋力
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 28640.7418769258
R2: 0.8779128035444631


In [20]:
# 用回歸模型補充異常光照度對應的發電量
data.loc[data['Sunlight(Lux)'] == 117758.2, 'Power(mW)'] = \
    model.predict(data[data['Sunlight(Lux)'] == 117758.2][['Sunlight(Lux)']])

In [21]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1290889,9,2024-07-23 15:50:57.000,0.0,994.54,30.69,72.91,2288.33,1.10
1290890,9,2024-07-23 15:51:57.000,0.0,994.40,30.27,73.16,3236.67,1.92
1290891,9,2024-07-23 15:52:57.000,0.0,994.39,29.90,72.51,4526.67,3.57
1290892,9,2024-07-23 15:53:57.000,0.0,994.40,29.38,73.23,4231.67,3.13


# Model Training

**use Lstm and pre-processed data to train our model**